<a href="https://colab.research.google.com/github/MathewBiddle/ioos_metrics/blob/yearly_ra_gts/notebooks/IOOS_obs_2_NDBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install erddapy

In [2]:
import pandas as pd
from erddapy import ERDDAP

# Function to grab the data.

In [3]:
def get_ndbc_full_stats():
    e = ERDDAP(
        server="https://erddap.ioos.us/erddap",
        protocol="tabledap",
    )
    e.response = "csv"
    dsets = {"IOOS": "gts_regional_statistics",
             "NDBC": "gts_ndbc_statistics",
             "non-NDBC": "gts_non_ndbc_statistics"}
    df_out = pd.DataFrame()
    for key, value in dsets.items():
        e.dataset_id = value
        df = e.to_pandas(
            index_col="time (UTC)",
            parse_dates=True
        )
        df["source"] = key
        df_out = pd.concat([df_out,df])
    return df_out

# Go get the data and return the dataframe

In [4]:
df = get_ndbc_full_stats()

df

,Year,Month,locationID,region,sponsor,met,wave,source
time (UTC),,,,,,,,
2018-01-01 00:00:00+00:00,2018,1,46108,AOOS,ALASKA OCEAN OBSERVING SYSTEM,0,2592,IOOS
2018-01-01 00:00:00+00:00,2018,1,AJXA2,AOOS,MARINE EXCHANGE OF ALASKA,8796,0,IOOS
2018-01-01 00:00:00+00:00,2018,1,CDXA2,AOOS,MARINE EXCHANGE OF ALASKA,4782,0,IOOS
2018-01-01 00:00:00+00:00,2018,1,ERXA2,AOOS,MARINE EXCHANGE OF ALASKA,5634,0,IOOS
2018-01-01 00:00:00+00:00,2018,1,GIXA2,AOOS,MARINE EXCHANGE OF ALASKA,8798,0,IOOS
...,...,...,...,...,...,...,...,...
2025-03-01 00:00:00+00:00,2025,3,OCSM2,NaN,U.S. ARMY CORPS OF ENGINEERS,0,0,non-NDBC
2025-03-01 00:00:00+00:00,2025,3,44097,NaN,U.S. ARMY CORPS OF ENGINEERS,0,2976,non-NDBC
2025-03-01 00:00:00+00:00,2025,3,FRFN7,NaN,U.S. ARMY CORPS OF ENGINEERS,0,0,non-NDBC


# Compute yearly totals of number of IOOS messages sent to the GTS.

In [5]:
yearly_totals = df.groupby(by=["source", pd.Grouper(freq="YE")])[['met','wave']].sum()
print(yearly_totals.loc['IOOS'].to_markdown(floatfmt=''))

| time (UTC)                |        met |      wave |
|:--------------------------|-----------:|----------:|
| 2018-12-31 00:00:00+00:00 | 10250188.0 | 1586648.0 |
| 2019-12-31 00:00:00+00:00 | 11344260.0 | 1682754.0 |
| 2020-12-31 00:00:00+00:00 | 10501536.0 | 1410472.0 |
| 2021-12-31 00:00:00+00:00 | 10189996.0 | 1610856.0 |
| 2022-12-31 00:00:00+00:00 | 11295426.0 | 1616542.0 |
| 2023-12-31 00:00:00+00:00 | 12719038.0 | 1796258.0 |
| 2024-12-31 00:00:00+00:00 | 12794686.0 | 1726010.0 |
| 2025-12-31 00:00:00+00:00 |  2910610.0 |  338230.0 |


# Compute yearly totals of IOOS messages sent to the GTS by region.

In [6]:
yearly_region_totals = df.groupby(by=["source", "region", pd.Grouper(freq="YE")])[['met','wave']].sum()
yearly_region_totals.reset_index('region',inplace=True)
print(yearly_region_totals.loc['IOOS'].to_markdown(floatfmt=''))

| time (UTC)                | region   |     met |   wave |
|:--------------------------|:---------|--------:|-------:|
| 2018-12-31 00:00:00+00:00 | AOOS     | 2571070 |  27968 |
| 2019-12-31 00:00:00+00:00 | AOOS     | 2810422 |  31584 |
| 2020-12-31 00:00:00+00:00 | AOOS     | 2697606 |   8258 |
| 2021-12-31 00:00:00+00:00 | AOOS     | 2574888 |  24336 |
| 2022-12-31 00:00:00+00:00 | AOOS     | 3675118 |  37414 |
| 2023-12-31 00:00:00+00:00 | AOOS     | 5620044 |  34026 |
| 2024-12-31 00:00:00+00:00 | AOOS     | 5939434 |  31992 |
| 2025-12-31 00:00:00+00:00 | AOOS     | 1516796 |   8100 |
| 2018-12-31 00:00:00+00:00 | CARICOOS |  619422 |  68520 |
| 2019-12-31 00:00:00+00:00 | CARICOOS |  719876 |  83582 |
| 2020-12-31 00:00:00+00:00 | CARICOOS |  629990 |  93232 |
| 2021-12-31 00:00:00+00:00 | CARICOOS |  676958 |  99074 |
| 2022-12-31 00:00:00+00:00 | CARICOOS |  506648 | 101264 |
| 2023-12-31 00:00:00+00:00 | CARICOOS |  335790 | 120508 |
| 2024-12-31 00:00:00+00:00 | CARICOOS |

# Calculate the percentage of IOOS OBS per year

In [7]:
yearly_totals = df.groupby(by=["source", pd.Grouper(freq="YE")])[['met','wave']].sum()

pcnt_obs_year = yearly_totals.loc['IOOS'] / (yearly_totals.loc['IOOS'] + yearly_totals.loc['NDBC'] + yearly_totals.loc['non-NDBC'])

print(pcnt_obs_year.to_markdown())

| time (UTC)                |      met |     wave |
|:--------------------------|---------:|---------:|
| 2018-12-31 00:00:00+00:00 | 0.155619 | 0.296562 |
| 2019-12-31 00:00:00+00:00 | 0.167812 | 0.312697 |
| 2020-12-31 00:00:00+00:00 | 0.154613 | 0.277295 |
| 2021-12-31 00:00:00+00:00 | 0.149813 | 0.3107   |
| 2022-12-31 00:00:00+00:00 | 0.153994 | 0.278716 |
| 2023-12-31 00:00:00+00:00 | 0.167084 | 0.222074 |
| 2024-12-31 00:00:00+00:00 | 0.167279 | 0.217845 |
| 2025-12-31 00:00:00+00:00 | 0.173252 | 0.228609 |


# Total number of unique platforms submitting data to the GTS from the IOOS regions

In [8]:
df.loc[df['source']=='IOOS'].locationID

,locationID
time (UTC),
2018-01-01 00:00:00+00:00,46108
2018-01-01 00:00:00+00:00,AJXA2
2018-01-01 00:00:00+00:00,CDXA2
2018-01-01 00:00:00+00:00,ERXA2
2018-01-01 00:00:00+00:00,GIXA2
...,...
2025-03-01 00:00:00+00:00,SIPF1
2025-03-01 00:00:00+00:00,42098
2025-03-01 00:00:00+00:00,44095


In [9]:
len(df.loc[df['source']=='IOOS'].locationID.unique())

260

# Total number of sponsors submitting data to the GTS via IOOS Regions

In [10]:
df.loc[df['source']=='IOOS'].sponsor

,sponsor
time (UTC),
2018-01-01 00:00:00+00:00,ALASKA OCEAN OBSERVING SYSTEM
2018-01-01 00:00:00+00:00,MARINE EXCHANGE OF ALASKA
2018-01-01 00:00:00+00:00,MARINE EXCHANGE OF ALASKA
2018-01-01 00:00:00+00:00,MARINE EXCHANGE OF ALASKA
2018-01-01 00:00:00+00:00,MARINE EXCHANGE OF ALASKA
...,...
2025-03-01 00:00:00+00:00,FLORIDA INSTITUTE OF TECHNOLOGY
2025-03-01 00:00:00+00:00,GREATER TAMPA BAY MARINE ADVISORY COUNCIL PORTS
2025-03-01 00:00:00+00:00,UNIVERSITY OF NORTH CAROLINA COASTAL STUDIES


In [11]:
len(df.loc[df['source']=='IOOS'].sponsor.unique())

47